In [1]:
from src.lexer.hulk_lexer import HulkLexer
from src.hulk_grammar.hulk_grammar import G
from src.parsing import LR1Parser


lexer = HulkLexer()
parser = LR1Parser(G)

In [2]:
from src.evaluation import evaluate_reverse_parse
from src.semantics.semantic_analysis_pipeline import semantic_analysis_pipeline
from src.semantics.formatter_visitor import FormatterVisitor

tokens, errors = lexer('''
    type C (cc : Number) {
        x = cc;
    }

    type B inherits C {
        f : Number = cc * cc / cc;
        g : String = "qwerty";
                               
        getsum(s: Number) : String => print(self.f);
    }

    type A(q : Number, r : Number) inherits B (q * r + 5) {
        f : Number = q;
        p : Number = r;
                       
        getsum(s: Number) : String => print(self.f);
    }

    function operate(x : Number, y : Object) : String {
        print((x + (y as Number)) as Object);
    }
                       
    function lambda(a : A) {
        print(a);
    }
                       
    let a : Number = 6 in
    let b : Number = a * 7, c : Object = b in
        lambda(new A(a, b));
    '''
)

derivation, operations = parser([t.token_type for t in tokens])
ast = evaluate_reverse_parse(derivation, operations, tokens)
ast, errors, context, scope = semantic_analysis_pipeline(ast, debug=False)

In [3]:
from src.code_gen.expression_visitor import CodeGenC

def getlinesindented(code: str):
    lines = ["   " + line for line in code.split('\n') if len(line.strip(' ')) > 0]
    return '\n'.join(lines)
    

codgen = CodeGenC(context)

code = ""

create_defs = {}
method_defs = {}
function_defs = []

for type in context.types.values():
    if type.name not in ["Number", "Bool", "String", "Object"]:
        create_def = "Object* create" + type.name + " ("
        create_params = []

        current = type

        while current is not None:
            for att in current.attributes:
                create_params.append(current.name + "_" + att.name)
                create_def += "Object* " + create_params[-1] + ", "

            current = current.parent

        if len(create_params) > 0:
            create_def = create_def[:-2]
        
        create_def += ")"

        create_defs[type.name] = (create_def, create_params)

        code += create_def + ";\n"

        method_defs[type.name] = []

        for method in type.methods:
            method_name = "method_" + type.name + "_" + method.name
            method_def = "Object* " + method_name + " (Object* self"
            
            method.node.scope.children[0].find_variable("self").setNameC("self")

            for i, name in enumerate(method.param_names):
                id_param = "p" + str(i)
                method.node.scope.children[0].find_variable(name).setNameC(id_param)
                method_def += ", Object* " + id_param

            method_def += ")"
            method_defs[type.name].append((method_def, method_name, method))
            code += method_def + ";\n"
                
        code += "\n"

for function in context.functions.values():
    if function.name not in ['print', 'sqrt', 'sin', 'cos', 'exp', 'log', 'rand']:
            function_name = "function_" + function.name
            function_def = "Object* " + function_name + " ("
            
            for i, name in enumerate(function.param_names):
                id_param = "p" + str(i)
                function.node.scope.children[0].find_variable(name).setNameC(id_param)
                function_def += "Object* " + id_param + ", "

            if len(function.param_names):
                function_def = function_def[:-2]

            function_def += ")"
            function_defs.append((function_def, function_name, function))
            code += function_def + ";\n"

code += '\n'

for type in context.types.values():
    if type.name not in ["Number", "Bool", "String", "Object"]:
        code += create_defs[type.name][0] + " {\n"
        code += "   Object* obj = createObject();\n"
        code += "   addAttribute(obj, \"type\",\"" + type.name + "\");\n"

        code += "\n"
        for param in create_defs[type.name][1]:
            code += "   addAttribute(obj, \"" + param + "\", " + param + ");\n"

        code += "\n"

        current = type

        while current is not None and current.name != "Object":
            if current.name in method_defs:
                for method in method_defs[current.name]:
                    code += "   addAttribute(obj, \"" + method[1] + "\", *" + method[1] + ");\n"

            current = current.parent

        code += "}\n\n"

for type in context.types.values():
    if type.name not in ["Number", "Bool", "String", "Object"]:
        if type.name in method_defs:
            for method_def, method_name, method in method_defs[type.name]:
                code += method_def + " {\n"
                code += getlinesindented(codgen.visit(method.node)[0]) + "\n"
                code += "}\n\n"
        
for function_def, function_name, function in function_defs:
    code += function_def + " {\n"
    code += getlinesindented(codgen.visit(function.node)[0]) + "\n"
    code += "}\n\n"

code += "\nint main() {\n"

code += getlinesindented(codgen.visit(ast.expression)[0]) + "\n"

code += "   return 0; \n}"

print(code)

Object* createC (Object* C_x);

Object* createB (Object* B_f, Object* B_g, Object* C_x);
Object* method_B_getsum (Object* self, Object* p0);

Object* createA (Object* A_f, Object* A_p, Object* B_f, Object* B_g, Object* C_x);
Object* method_A_getsum (Object* self, Object* p0);

Object* function_operate (Object* p0, Object* p1);
Object* function_lambda (Object* p0);

Object* createC (Object* C_x) {
   Object* obj = createObject();
   addAttribute(obj, "type","C");

   addAttribute(obj, "C_x", C_x);

}

Object* createB (Object* B_f, Object* B_g, Object* C_x) {
   Object* obj = createObject();
   addAttribute(obj, "type","B");

   addAttribute(obj, "B_f", B_f);
   addAttribute(obj, "B_g", B_g);
   addAttribute(obj, "C_x", C_x);

   addAttribute(obj, "method_B_getsum", *method_B_getsum);
}

Object* createA (Object* A_f, Object* A_p, Object* B_f, Object* B_g, Object* C_x) {
   Object* obj = createObject();
   addAttribute(obj, "type","A");

   addAttribute(obj, "A_f", A_f);
   addAttribute(o